In [14]:
import numpy as np

ideal_point = np.array([0, 0])  
nadir_point = np.array([10, 10])

divisions_per_dim = 2
grid_intervals = (nadir_point - ideal_point) / divisions_per_dim
solutions = np.array([
    [1, 2],
    [9, 8],
    [5, 5],
    [3, 9]
])

def assign_solutions_to_subspaces(solutions, ideal_point, grid_intervals):
    relative_positions = solutions - ideal_point
    subspace_indices = np.floor(relative_positions / grid_intervals).astype(int)
    return subspace_indices

subspace_indices = assign_solutions_to_subspaces(solutions, ideal_point, grid_intervals)

array([[1, 1],
       [0, 0],
       [1, 1],
       [1, 0]])

In [1]:
import numpy as np
import random

class AGMOEA:
    def __init__(self, NP, K, NGBA, NEXA):
        self.NP = NP
        self.K = K
        self.NGBA = NGBA
        self.NEXA = NEXA
        self.EXA = []  # External archive
        self.GBA = [[] for _ in range(K)]  # Subspace grid

    def initialize_parameters(self):
        # Initialize parameters such as population size (NP), interval number of each objective (K),
        # maximum size of each subspace (NGA), and maximum size of the external archive (NEXA)
        pass

    def construct_subspaces(self, solutions):
        # Calculate the grid intervals based on the ideal and nadir points
        grid_intervals = (self.nadir_point - self.ideal_point) / self.K

        # Something is wrong here!!!!
        # Reset the grid for the new generation
        self.GBA = [Subspace(np.full_like(self.ideal_point, i)) for i in range(self.K)]

        # Assign each solution to the appropriate subspace
        for solution in solutions:
            # Calculate the relative position of the solution in the objective space
            relative_position = solution.objective_values - self.ideal_point
            # Determine the grid coordinates
            grid_coordinates = np.floor(relative_position / grid_intervals).astype(int)
            # Find the corresponding subspace and add the solution to it
            self.GBA[tuple(grid_coordinates)].solutions.append(solution)


    def improve_EXA(self):
        # Improve the external archive EXA using the extension mechanism described in Section 3.6
        pass

    def SR(self, subspace):
        """
        Calculate the Subspace Ranking (SR) based on the summation of its coordinates.
        :param subspace: A Subspace instance.
        :return: The SR value as the sum of the subspace's coordinates.
        """
        # Assuming subspace.grid_coordinates is a list or array of coordinates
        return sum(subspace.grid_coordinates)

  
    def select_subspace(self):
        """
        Selects a subspace with an adaptive strategy.

        :param G: The set of all subspaces.
        :param S: The set of degraded subspaces, which is empty at the beginning of each generation.
        :return: The selected subspace and the updated set of degraded subspaces.
        """
        epsilon = 1e-6  # A small constant to prevent division by zero
        G = self.GBA  # Assuming GBA is the current set of all subspaces
        S = set()  # Assuming S is to be updated within this method

        probabilities = {k: (1 / (self.SR(k) + epsilon)) for k in G}
        total = sum(probabilities.values())
        probabilities = {k: (v / (total + len(G) * epsilon)) for k, v in probabilities.items()}

        # Select a subspace based on the calculated probabilities
        selected_subspace = random.choices(G, weights=probabilities.values(), k=1)[0]

        # Update the set of degraded subspaces based on the selection
        for subspace in G:
            if self.SR(selected_subspace) < self.SR(subspace):
                S.add(subspace)

        # Assuming S needs to be processed further or used elsewhere after this selection
        return selected_subspace, S

    def generate_offspring(self, subspace):
        # Generate an offspring considering the random or representative individual in the selected subspace
        pass

    def evaluate_individual(self, individual):
        # Evaluate the fitness of an individual
        pass

    def fast_non_dominated_sort(self, population):
        # Sort the population based on non-domination criteria
        pass

    def agmoea_algorithm(self):
        # Main procedure of the AGMOEA algorithm
        self.initialize_parameters()

        # Generate initial population P0
        P0 = []  # Initial population

        # Evaluate individuals in P0
        for individual in P0:
            self.evaluate_individual(individual)

        # Store non-dominated solutions in P0 into EXA
        self.EXA.extend(self.fast_non_dominated_sort(P0))

        # Main loop
        while not self.termination_criterion():
            # Construct subspaces
            self.construct_subspaces()

            # Improve EXA
            self.improve_EXA()

            # Set TP to be empty
            TP = []  # Temporary population

            # Generate offsprings and evaluate
            for _ in range(NP):
                # Select a subspace
                selected_subspace, S = self.select_subspace(G, S)
                
                # Generate an offspring
                offspring = self.generate_offspring(subspace)
                
                # Evaluate offspring and store in TP
                self.evaluate_individual(offspring)
                TP.append(offspring)

            # Update EXA with TP
            self.EXA.extend(self.fast_non_dominated_sort(TP))

            # Update population P with TP
            P = TP

            # Set P to be the set of the best NP individuals in P based on fast non-dominated sorting
            P = self.fast_non_dominated_sort(P)[:NP]

    def termination_criterion(self):
        # Define the termination criterion for the algorithm
        pass

